In [ ]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    get_bigquery_client,
    bq_table_exists,
    get_max_part,
    load_query_result_to_table,
    pandas_to_bq,
    pandas_to_bq_table,
    load_bigquery_ipython_magic,
    get_bigquery_client,
    _print_query_job_results,
    load_query_result_to_partitions
    
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    row_number, 
    col, 
    lit, 
    count, 
    log, 
    exp, 
    sum as spark_sum
)
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

In [ ]:
from google.cloud.bigquery.job import QueryJobConfig

In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta

In [ ]:
print(f'current_dt: {current_dt}')
print(f'state: {state}')
print(f'log_duration: {log_duration}')

In [ ]:
# 최신날짜 계산
execution_dt = datetime.strptime(current_dt, '%Y-%m-%d')
execution_dt_one_ago = (execution_dt - timedelta(days=1))
lag_current_dt= execution_dt_one_ago.strftime('%Y-%m-%d')
print(f'execution_dt: {lag_current_dt}')

In [ ]:
log_duration = int(log_duration) - 1
short_duration = 6

In [ ]:
end_dt = lag_current_dt
start_dt = (execution_dt_one_ago - timedelta(days=log_duration)).strftime('%Y-%m-%d')
print("long_start_dt : ", end_dt)
print("short_start_dt : ", start_dt)

In [ ]:
db_name = 'adot_reco_dev'
table_nm = 'tdeal_cat1_cnt'
project_id = 'skt-datahub'

In [ ]:
bq_client = get_bigquery_client()

# 운영성

In [ ]:
tdeal_temp_cat_list = {
    "여행/티켓": [],
    "건강식품": [],
    "스포츠/레저": [],
    "출산/육아": [],
    "반려동물용품": [],
    "화장품/미용": [],
    "디지털/가전": []
}

In [ ]:
tdeal_where_clauses = []
for key, vals in tdeal_temp_cat_list.items():
    tdeal_where_clauses.append(
        f"TRIM(LOWER(cat2)) = '{key}'"
    )
    tdeal_where_clauses.append("OR")
tdeal_where_clause = ' '.join(tdeal_where_clauses[:-1])

In [ ]:
# 상대 비교 프로파일 추출 |로직
cat1_query = f"""
WITH params AS (
  SELECT
    {long_duration/2} AS longterm_decay_param, 
    {short_duration/2} AS shortterm_decay_param
),

BUY_TABLE AS (
    SELECT distinct svc_mgmt_num, 
                    luna_id,
                    cat2, 
                    cat1, 
                    cat3, 
                    is_weekend, 
                    dt
    
    FROM {db_name}.{table_nm}
    WHERE dt <= '{lag_current_dt}' 
    AND dt >= '{start_dt}'
    AND cat3='buy'
    AND ( {tdeal_where_clause} )
),

USER_PURCHASE_TABLE AS (

    SELECT  *
           
    FROM (
        SELECT  svc_mgmt_num,
                luna_id,
                cat2,
                cat1,
                ARRAY_AGG(dt ORDER BY dt asc) as purchase_date
        FROM BUY_TABLE
        group by svc_mgmt_num, luna_id, cat1, cat2
    )
),
unnested_purchases AS (
  SELECT
    svc_mgmt_num,
    luna_id,
    cat2,
    cat1,
    purchase_date
    
  FROM USER_PURCHASE_TABLE, UNNEST(purchase_date) AS purchase_date
),

purchase_metrics AS (
    SELECT  svc_mgmt_num,
            luna_id,
            cat1,
            cat2,
            COUNT(*) AS frequency,
            ARRAY_AGG(DATE_DIFF(PARSE_DATE('%Y-%m-%d', '{lag_current_dt}'), purchase_date, DAY) ORDER BY DATE_DIFF(PARSE_DATE('%Y-%m-%d', '{lag_current_dt}'), purchase_date, DAY) DESC) AS days_since_purchases
           
  FROM unnested_purchases
  GROUP BY svc_mgmt_num, luna_id, cat1, cat2
),

interest_score AS (
  SELECT
    svc_mgmt_num,
    luna_id,
    cat1,
    cat2,
    frequency,
    (
      SELECT SUM(EXP(-1 * day / longterm_decay_param))  -- Exponential decay with 60-day half-life
      FROM UNNEST(days_since_purchases) day
    ) AS long_term_recency_score,
    (
      SELECT SUM(EXP(-1 * day / shortterm_decay_param))  
      FROM UNNEST(days_since_purchases) day
    ) AS short_term_recency_score,
  FROM purchase_metrics, params
)

SELECT *
FROM interest_score
"""

In [ ]:
job_config = QueryJobConfig()
temp_table = f'{project_id}.adot_reco_dev.jh_tdeal_eda_temp'
job_config.destination = temp_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [ ]:
# 절대 비교 프로파일 추출 로직
query_job = bq_client.query(cat1_query, job_config=job_config)
query_job.result() 

In [ ]:
query = f"""
SELECT  cat1,
        cat2,
        APPROX_QUANTILES(long_term_recency_score, 4) as long_percentile_list,
        APPROX_QUANTILES(short_term_recency_score, 4) as short_percentile_list,
FROM {temp_table}
GROUP BY cat1, cat2
"""

In [ ]:
percentile_table_cat1 = bq_to_pandas(query)

In [ ]:
long_quantile_tables_cat1 = percentile_table_cat1.set_index('cat1').to_dict()['long_percentile_list']
short_quantile_tables_cat1 = percentile_table_cat1.set_index('cat1').to_dict()['short_percentile_list']

In [ ]:
query = f"""
SELECT distinct cat2, 
                cat1, 
                cat3, 
                dt

FROM {db_name}.{table_nm}
WHERE dt <= '{end_dt}' 
AND dt >= '{start_dt}'
AND cat3='buy'
AND ( {tdeal_where_clause} )
"""

In [ ]:
cat_df = bq_to_pandas(query)

In [ ]:
cat_dict = cat_df.set_index("cat1").to_dict()['cat2']

In [ ]:
cat1_list = ["스포츠/레저"]    

In [ ]:
long_percentile_cat1_conditions = " OR ".join([f"(cat1 = '{key}' AND long_term_recency_score>={values[3]})" for key, values in long_quantile_tables_cat1.items() if cat_dict[key] in cat1_list])
short_percentile_cat1_conditions = " OR ".join([f"(cat1 = '{key}' AND long_term_recency_score>={values[3]})" for key, values in short_quantile_tables_cat1.items() if cat_dict[key] in cat1_list])

In [ ]:
cat1_query = f"""
    SELECT  *,
            "long" as pref
    FROM {temp_table}
    WHERE ({long_percentile_cat1_conditions})
    
    UNION ALL
    SELECT  *,
            "short" as pref
    FROM {temp_table}
    WHERE ({short_percentile_cat1_conditions})
"""

In [ ]:
temp_table = f'{project_id}.adot_reco_dev.jh_tdeal_cat1_temp'

In [ ]:
job_config = QueryJobConfig()
job_config.destination = temp_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [ ]:
# 절대 비교 프로파일 추출 로직
query_job = bq_client.query(cat1_query, job_config=job_config)
query_job.result() 

# CAT2

In [ ]:
# 상대 비교 프로파일 추출 로직
cat2_query = f"""
WITH params AS (
  SELECT
    {long_duration/2} AS longterm_decay_param, 
    {short_duration/2} AS shortterm_decay_param
),

BUY_TABLE AS (
    SELECT distinct svc_mgmt_num, 
                    luna_id,
                    cat2, 
                    is_weekend, 
                    dt
    
    FROM {db_name}.{table_nm}
    WHERE dt <= '{end_dt}' 
    AND dt >= '{start_dt}'
    AND cat3='buy'
    AND ( {tdeal_where_clause} )
),

USER_PURCHASE_TABLE AS (

    SELECT  *
           
    FROM (
        SELECT  svc_mgmt_num,
                luna_id,
                cat2,
                ARRAY_AGG(dt ORDER BY dt asc) as purchase_date
        FROM BUY_TABLE
        group by svc_mgmt_num, luna_id, cat2
    )
),
unnested_purchases AS (
  SELECT
    svc_mgmt_num,
    luna_id,
    cat2,
    purchase_date
    
  FROM USER_PURCHASE_TABLE, UNNEST(purchase_date) AS purchase_date
),

purchase_metrics AS (
    SELECT  svc_mgmt_num,
            luna_id,
            cat2,
            COUNT(*) AS frequency,
            ARRAY_AGG(DATE_DIFF(PARSE_DATE('%Y-%m-%d', '{end_dt}'), purchase_date, DAY) ORDER BY DATE_DIFF(PARSE_DATE('%Y-%m-%d', '{end_dt}'), purchase_date, DAY) DESC) AS days_since_purchases
           
  FROM unnested_purchases
  GROUP BY svc_mgmt_num, luna_id, cat2
),

interest_score AS (
  SELECT
    svc_mgmt_num,
    luna_id,
    cat2,
    frequency,
    (
      SELECT SUM(EXP(-1 * day / longterm_decay_param))  -- Exponential decay with 60-day half-life
      FROM UNNEST(days_since_purchases) day
    ) AS long_term_recency_score,
    (
      SELECT SUM(EXP(-1 * day / shortterm_decay_param))  
      FROM UNNEST(days_since_purchases) day
    ) AS short_term_recency_score,
  FROM purchase_metrics, params
)

SELECT *
FROM interest_score
"""

In [ ]:
temp_table = f'{project_id}.adot_reco_dev.jh_tdeal_eda_temp'
job_config = QueryJobConfig()
job_config.destination = temp_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [ ]:
# 절대 비교 프로파일 추출 로직
query_job = bq_client.query(cat2_query, job_config=job_config)
query_job.result() 

In [ ]:
query = f"""
SELECT  cat2,
        APPROX_QUANTILES(long_term_recency_score, 4) as long_percentile_list,
        APPROX_QUANTILES(short_term_recency_score, 4) as short_percentile_list,
FROM {temp_table}
GROUP BY cat2
"""

In [ ]:
percentile_table_cat2 = bq_to_pandas(query)

In [ ]:
long_quantile_tables_cat2 = percentile_table_cat2.set_index('cat2').to_dict()['long_percentile_list']
short_quantile_tables_cat2 = percentile_table_cat2.set_index('cat2').to_dict()['short_percentile_list']

In [ ]:
cat2list = list(set(cat_dict.values()))

In [ ]:
user_state_list = ["반려동물용품", "출산/육아"]

In [ ]:
long_percentile_conditions = " OR ".join([f"(cat2 = '{key}' AND long_term_recency_score>={values[3]})" if key not in user_state_list else f"(cat2 = '{key}' AND long_term_recency_score>={values[1]})"  for key, values in long_quantile_tables_cat2.items()])
short_percentile_conditions = " OR ".join([f"(cat2 = '{key}' AND short_term_recency_score>={values[3]})" if key not in user_state_list else f"(cat2 = '{key}' AND short_term_recency_score>={values[1]})"  for key, values in short_quantile_tables_cat2.items()])

In [ ]:
cat2_query = f"""
    SELECT  *,
            "long" as pref
    FROM {temp_table}
    WHERE ({long_percentile_conditions})
    
    UNION ALL
    SELECT  *,
            "short" as pref
    FROM {temp_table}
    WHERE ({short_percentile_conditions})
"""

In [ ]:
temp_table = f'{project_id}.adot_reco_dev.jh_tdeal_cat2_temp'

In [ ]:
job_config = QueryJobConfig()
job_config.destination = temp_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [ ]:
# 절대 비교 프로파일 추출 로직
query_job = bq_client.query(cat2_query, job_config=job_config)
query_job.result() 

In [ ]:
query = f"""
WITH User_state_tables AS(
    SELECT  svc_mgmt_num,
            luna_id,
            ARRAY_AGG(user_state) as user_state,
    FROM (
        SELECT distinct svc_mgmt_num,
                        luna_id,
                        CASE
                            WHEN cat2 = "출산/육아" THEN '육아/아기'
                            ELSE '반려동물'
                        END AS user_state

        FROM skt-datahub.adot_reco_dev.jh_tdeal_cat2_temp
        WHERE cat2 in ("출산/육아", "반려동물용품")
    )
    GROUP BY svc_mgmt_num, luna_id
),

Long_User_preference_tables AS (
        SELECT  svc_mgmt_num,
                luna_id,
                ARRAY_AGG(long_prefered_domain) as long_prefered_domain,
        FROM (
            SELECT distinct  svc_mgmt_num,
                             luna_id,
                             cat2 as long_prefered_domain

            FROM skt-datahub.adot_reco_dev.jh_tdeal_cat2_temp
            WHERE cat2 not in ("출산/육아", "반려동물용품") and pref='long'

            UNION ALL

            SELECT distinct  svc_mgmt_num,
                             luna_id,
                             cat1 as long_prefered_domain

            FROM skt-datahub.adot_reco_dev.jh_tdeal_cat1_temp
            WHERE pref='long'
        ) 
        GROUP BY svc_mgmt_num, luna_id
),

Short_User_preference_tables AS (

    SELECT *
    FROM (
        SELECT  svc_mgmt_num,
                luna_id,
                ARRAY_AGG(short_prefered_domain) as short_prefered_domain,
        FROM (
            SELECT distinct  svc_mgmt_num,
                             luna_id,
                             cat2 as short_prefered_domain

            FROM skt-datahub.adot_reco_dev.jh_tdeal_cat2_temp
            WHERE cat2 not in ("출산/육아", "반려동물용품") and pref='short'

            UNION ALL

            SELECT distinct  svc_mgmt_num,
                             luna_id,
                             cat1 as short_prefered_domain

            FROM skt-datahub.adot_reco_dev.jh_tdeal_cat1_temp
            WHERE pref='short'
        ) 
        GROUP BY svc_mgmt_num, luna_id
    )
)

SELECT  A.svc_mgmt_num,
        A.luna_id,
        A.long_prefered_domain as  long_term_profile,
        A.short_prefered_domain as short_term_profile,
        B.user_state as user_state_profile,
        PARSE_DATE('%Y-%m-%d', '{lag_current_dt}') as dt 
FROM (
    SELECT 
    COALESCE(t1.svc_mgmt_num, t2.svc_mgmt_num) AS svc_mgmt_num,
    COALESCE(t1.luna_id, t2.luna_id) AS luna_id,
    t1.long_prefered_domain,
    t2.short_prefered_domain
    FROM Long_User_preference_tables t1
    FULL OUTER JOIN Short_User_preference_tables t2
    ON t1.svc_mgmt_num = t2.svc_mgmt_num
) AS A

LEFT JOIN (
    SELECT *
    FROM User_state_tables
)AS B
ON A.svc_mgmt_num = B.svc_mgmt_num

"""

# 프로 파일 테이블 저장

In [ ]:
PROJECT_ID = "skt-datahub"
db_name = "adot_reco_dev"
partitioned_dest_table = "adotServiceProfile_tdeal"

In [ ]:
table_exists = bq_table_exists(table=f'{db_name}.{partitioned_dest_table}', project_id = PROJECT_ID)

In [ ]:
if not table_exists:
    get_bigquery_client().query(f"""
        CREATE TABLE IF NOT EXISTS {db_name}.{partitioned_dest_table}(
            svc_mgmt_num STRING,
            luna_id STRING,
            long_term_profile STRING,
            short_term_profile STRING,
            adot_cat1_go_to_work STRING,
            user_state_profile STRING,
            dt DATE
        )
        PARTITION BY dt
    """).result()

    print(f"생성된 테이블 : {db_name}.{partitioned_dest_table}")

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{db_name}.{partitioned_dest_table}', partition='dt')

# Template 입히기

In [ ]:
template_query = f"""
    SELECT * 
    FROM adot_reco_dev.profile_template
    WHERE source_domain = 'tdeal'
"""

In [ ]:
template_df = bq_to_pandas(template_query)

In [ ]:
tdeal_long_template = template_df.loc[template_df.property=='long'].template.values[0]
tdeal_short_template = template_df.loc[template_df.property=='short'].template.values[0]
tdeal_state_template = template_df.loc[template_df.property=='state'].template.values[0]

In [ ]:
query = f"""
WITH LOW_TABLE AS (
    SELECT  luna_id,
            CASE 
                WHEN longterm_profile='' THEN ""
                ELSE    REGEXP_REPLACE(
                          '{tdeal_long_template}',
                          r'\\{{longterm_profile\\}}',
                          longterm_profile
                    )
            END AS longterm_profile_template,

            CASE 
                WHEN shortterm_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '{tdeal_short_template}',
                    r'\\{{shortterm_profile\\}}',
                        shortterm_profile
                    ) 
            END AS shortterm_profile_template,
            CASE 
                WHEN state_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '{tdeal_state_template}',
                    r'\\{{state_profile\\}}',
                        state_profile
                    ) 
            END AS state_profile_template,
            dt

    FROM (
        SELECT  luna_id,            
                COALESCE(ARRAY_TO_STRING(long_prefered_domain, ','), '') as longterm_profile,
                COALESCE(ARRAY_TO_STRING(short_prefered_domain, ','), '') as shortterm_profile,
                COALESCE(ARRAY_TO_STRING(user_state, ','), '') as state_profile,
                dt
        FROM {db_name}.{partitioned_dest_table}
        WHERE luna_id is not null
    )
),

MIDDLE_TABLE AS(
    SELECT  luna_id,
            CASE 
                WHEN longterm_profile_template !='' THEN  CONCAT(longterm_profile_template, ',' , shortterm_profile_template)
            ELSE shortterm_profile_template
            END AS tdeal_profile,
            state_profile_template,
            dt
    FROM LOW_TABLE
)

SELECT  luna_id,
        CASE 
            WHEN state_profile_template !='' THEN  CONCAT(state_profile_template, ',' , tdeal_profile)
            ELSE tdeal_profile
        END AS profile_templates,
        "tdeal" as source_domain,
        dt
FROM MIDDLE_TABLE
"""

In [ ]:
table_name = "adotServiceProfile_templated_tdeal"

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{db_name}.{table_name}', partition='dt')